In [ ]:
import pandas as pd
from prophet.plot import plot_plotly, plot_components_plotly
from prophet import Prophet
from statsmodels.tools.eval_measures import rmse

df = pd.read_csv('/Users/alanchan/Documents/UCL/Year 4/ELEC0088/SNS assignment 22-23/London Weather data/london_weather.csv')
print(df.info())
df.dropna(inplace= True)
df.reset_index(drop=True, inplace=True)
print(df.info())

In [ ]:
# print(df.tail())
df = df.rename(columns={'mean_temp': 'y', 'date':'ds'})
df = df[-2000:]
# df=df[["date","mean_temp","pressure"]]
# df.columns = ['ds','y',"pressure"]
df['ds'] = pd.to_datetime(df['ds'], format='%Y%m%d')
print(df.tail())

In [ ]:
df.sort_values(by='ds', inplace = True)
df.reset_index(drop=True, inplace=True)
# print(df.head())
print(df.tail())

In [ ]:
df.plot(x='ds',y='y',figsize=(18,6))
print(len(df))

In [ ]:
train = df.iloc[:len(df)-365]
test = df.iloc[len(df)-365:]

print(len(train))
print(len(test))

print(df.tail())

In [ ]:
def preprocessing (multi_var,periods):
    t = train[['ds',multi_var]]
    t = t.rename(columns={multi_var: 'y', 'date':'ds'})
    model = Prophet()
    model.fit(t)
    future = model.make_future_dataframe(periods=periods)
    forecast = model.predict(future)
    return forecast['yhat']

periods = 365*2

pressure = preprocessing('pressure',periods)
cloud_cover = preprocessing('cloud_cover',periods)
sunshine = preprocessing('sunshine',periods)
global_radiation = preprocessing('global_radiation',periods)
precipitation = preprocessing('precipitation',periods)
snow_depth = preprocessing('snow_depth',periods)

In [ ]:
m = Prophet()
m.add_regressor('pressure')
m.add_regressor('cloud_cover')
m.add_regressor('sunshine')
m.add_regressor('global_radiation')
m.add_regressor('precipitation')
m.add_regressor('snow_depth')

m.fit(train)

future = m.make_future_dataframe(periods=periods) #MS for monthly, H for hourly

# print(future.tail())
future['pressure'] = pressure
future['cloud_cover'] = cloud_cover
future['sunshine'] = sunshine
future['global_radiation'] = global_radiation
future['precipitation'] = precipitation
future['snow_depth'] = snow_depth

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
plot_plotly(m ,forecast)

In [ ]:
plot_components_plotly(m, forecast)

In [ ]:
predictions = forecast.iloc[-len(test):]['yhat']
print("Root Mean Squared Error between actual and  predicted values: ",rmse(predictions,test['y']))
print("Mean Value of Test Dataset:", test['y'].mean())

In [ ]:
training = forecast.iloc[:len(train)]['yhat']
print("Root Mean Squared Error between actual and  predicted values: ",rmse(training,train['y']))